In [1]:
import os
import shutil

train_filenames = os.listdir('Classified_by_period')
train_MIBC = filter(lambda x:x[:3] == 'MIBC', train_filenames)
train_NMIBC = filter(lambda x:x[:3] == 'NMIBC', train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

rmrf_mkdir('test2')
os.symlink('../test/', 'test2/test')

for filename in train_cat:
    os.symlink('../../train/'+filename, 'train2/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'train2/dog/'+filename)


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'train'

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train", image_size, shuffle=False, 
                                              batch_size=16)
    print(train_generator)
    test_generator = gen.flow_from_directory("train", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    print(test_generator)
    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test) 
        h.create_dataset("label", data=train_generator.classes)

#write_gap(NASNetLarge, (331, 331), nasnet.preprocess_input)       
write_gap(InceptionResNetV2, (299, 299), inception_resnet_v2.preprocess_input)
#write_gap(Xception, (299, 299), xception.preprocess_input)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Found 1320 images belonging to 2 classes.
Found 1320 images belonging to 2 classes.


In [2]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:    
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [3]:
from keras.models import *
from keras.layers import *

np.random.seed(2017)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=64, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 382 samples, validate on 96 samples
Epoch 1/64
382/382 [==============================] - 1s - loss: 0.9184 - acc: 0.4921 - val_loss: 0.6157 - val_acc: 0.6771
Epoch 2/64
382/382 [==============================] - 0s - loss: 0.7254 - acc: 0.5995 - val_loss: 0.5415 - val_acc: 0.7083
Epoch 3/64
382/382 [==============================] - 0s - loss: 0.5869 - acc: 0.6623 - val_loss: 0.4833 - val_acc: 0.8125
Epoch 4/64
382/382 [==============================] - 0s - loss: 0.6078 - acc: 0.6702 - val_loss: 0.4345 - val_acc: 0.8229
Epoch 5/64
382/382 [==============================] - 0s - loss: 0.5081 - acc: 0.7277 - val_loss: 0.3981 - val_acc: 0.8125
Epoch 6/64
382/382 [==============================] - 0s - loss: 0.4652 - acc: 0.7853 - val_loss: 0.3649 - val_acc: 0.9167
Epoch 7/64
382/382 [==============================] - 0s - loss: 0.4332 - acc: 0.8010 - val_loss: 0.3371 - val_acc: 0.9479
Epoch 8/64
382/382 [==========================

In [13]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv",sep=",")


gen = ImageDataGenerator()

test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=10, class_mode=None)
print(len(test_generator.filenames))
print(len(y_pred))
for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.set_value(index-3, 'label', y_pred[i])

print(df)
df.to_csv('pred.csv', index=None)
df.head(10)
print('Done')

 32/478 [=>............................] - ETA: 0sFound 478 images belonging to 2 classes.
478
478
      id     label
0      3  0.009443
1      4  0.012548
2      5  0.007472
3      6  0.005000
4      7  0.005000
5      8  0.005000
6      9  0.005000
7     10  0.008037
8     11  0.009911
9     12  0.010580
10    13  0.020686
11    14  0.005000
12    15  0.010437
13    16  0.006839
14    17  0.006055
15    18  0.009612
16    19  0.007029
17    20  0.040653
18    21  0.028421
19    22  0.037276
20    23  0.007314
21    24  0.021139
22    25  0.005779
23    26  0.005330
24    27  0.016898
25    28  0.009382
26    29  0.008593
27    30  0.005000
28    31  0.011291
29    32  0.005000
..   ...       ...
448  451  0.005000
449  452  0.005000
450  453  0.005000
451  454  0.005000
452  455  0.005000
453  456  0.005000
454  457  0.005000
455  458  0.005000
456  459  0.005000
457  460  0.010910
458  461  0.011397
459  462  0.006899
460  463  0.005000
461  464  0.005000
462  465  0.005000
463  466

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
